In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pysr import PySRRegressor

In [2]:
def error(data,model):
    if len(data) != len(model):
        print('ERROR: array must have the same length!')
        return
    
    l = len(data)
    error = []
    
    for i in range(l):
        error.append(100*abs((data[i] - model[i]) / data[i]))
    
    return np.array(error)

In [3]:
def accuracy(data, model):
    if len(data) != len(model):
        print('ERROR: array must have the same length!')
        return
    
    l = len(data)
    total_error = 0
    
    for i in range(l):
        error = abs((data[i] - model[i]) / data[i])
        total_error += error
    
    avg_error = total_error / l
    accuracy_percentage = 100 * (1 - avg_error)
    
    return accuracy_percentage

In [4]:
model = PySRRegressor()

In [5]:
def bran(x,T,alpha):
    a = model.from_file(alpha)
    T_1 = a.predict(x)
    print('Model equation is: ', a.latex())
    fig, axs = plt.subplots(1, 2, figsize=(18, 6))
    axs[0].scatter(x,T, s=2, label='CLASS data')
    axs[0].plot(x[0:114], T_1[0:114], lw=1.5, label='From PySR', color='r')
    axs[0].set_xlabel(r'$x$')
    axs[0].set_ylabel(r'$T(x)$')
    axs[0].set_title('Computed vs SR')
    axs[0].loglog()
    axs[0].grid(True)
    axs[0].legend()


    axs[1].scatter(x,error(T,T_1),s=3)
    axs[1].set_xlabel(r'$x$')
    axs[1].set_ylabel(r'$\frac{|T_{CLASS}(x)-T_{SR}(x)|}{T_{CLASS}(x)}\cdot 100$')
    axs[1].set_title('Error')
    axs[1].grid(True)
    axs[1].loglog()

    print('The accuracy of the model is: ',np.round(accuracy(T,T_1),2),'%')
    print('Max percentage error is: ',np.round(np.amax(error(T,T_1)),2),'%')


lol

In [6]:
df = pd.read_csv('CL_6_50_sort.csv')
df

,tau,ell,EE,TE,TT
0,0.02,2,1.879053e-15,4.797379e-13,2.922889e-10
1,0.04,2,4.644890e-15,5.567004e-13,2.927408e-10
2,0.06,2,7.428302e-15,5.765040e-13,2.932656e-10
3,0.08,2,1.005212e-14,5.786188e-13,2.935428e-10
4,0.10,2,1.250106e-14,5.738810e-13,2.937351e-10
...,...,...,...,...,...
289,0.04,50,6.176447e-17,-7.801871e-16,6.287819e-13
290,0.06,50,5.937659e-17,-7.523330e-16,6.057582e-13
291,0.08,50,5.709061e-17,-7.227135e-16,5.842923e-13
292,0.10,50,5.493020e-17,-6.813153e-16,5.642719e-13


In [12]:
a = model.from_file('hall_of_fame_2023-09-14_154730.587.pkl')
a.latex()

Checking if hall_of_fame_2023-09-14_154730.587.pkl exists...
Loading model from hall_of_fame_2023-09-14_154730.587.pkl


'- 88.8 \\tau \\left(\\tau - 0.917\\right) - 0.611'

$$ 88.8 \tau^2 -80.7\tau - 0.611$$